In [1]:
from simpletransformers.language_modeling import LanguageModelingModel, LanguageModelingArgs
import pickle
import torch

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
import transformers
print(transformers.__version__)

3.0.2


In [2]:
train_args = {'fp16': False, 'use_multiprocessing': False, 'reprocess_input_data': False, 'overwrite_output_dir': True, 'num_train_epochs': 1, 'save_eval_checkpoints': False, 'save_model_every_epoch': True, 'learning_rate': 1e-05, 'warmup_steps': 1000, 'train_batch_size': 64, 'eval_batch_size': 64, 'gradient_accumulation_steps': 1, 'block_size': 130, 'max_seq_length': 130, 'dataset_type': 'simple', 'logging_steps': 500, 'evaluate_during_training': True, 'evaluate_during_training_steps': 500, 'evaluate_during_training_steps_anomaly': 500, 'anomaly_batch_size': 64, 'evaluate_during_training_verbose': True, 'use_cached_eval_features': True, 'sliding_window': True, 'vocab_size': 52000, 'eval_anomalies': True, 'random_generator': 1, 'use_rtd_loss': True, 'rtd_loss_weight': 50, 'rmd_loss_weight': 100, 'mlm_loss_weight': 1, 'dump_histogram': 0, 'eval_anomaly_after': 0, 'train_just_generator': 0, 'replace_tokens': 0, 'extract_scores': 1, 'subset_name': 'known', 'extract_repr': 0, 'train_document': True, 'tokenizer_name': 'bert-base-uncased', 'tensorboard_dir': 'runs/tests/known_slen128_wd0.1_lr1e-05-0.0001_msk50_p50_dl4_sz256_gl1_sz16_rgen1_drop0.5_w50_100_1_replace0_mlmr1_07232023_155706_cont0', 'extract_reps': 0, 'weight_decay': 0.1, 'optimizer': 'AdamW', 'scores_export_path': './token_scores/known_slen128_wd0.1_lr1e-05-0.0001_msk50_p50_dl4_sz256_gl1_sz16_rgen1_drop0.5_w50_100_1_replace0_mlmr1_07232023_155706_cont0/', 'generator_config': {'embedding_size': 128, 'hidden_size': 16, 'num_hidden_layers': 1}, 'discriminator_config': {'hidden_dropout_prob': 0.5, 'attention_probs_dropout_prob': 0.5, 'embedding_size': 128, 'hidden_size': 256, 'num_hidden_layers': 4}, 'mlm_lr_ratio': 1}

In [3]:
masks_ = pickle.load(open('/home/anhnmt2/Documents/CodeInjection/date/experiments/pseudo_labels128_p50.pkl', 'rb'))

In [4]:
model = LanguageModelingModel(
    "electra", 
    # '/home/anhnmt2/Documents/CodeInjection/date/experiments/outputs',
    None,
    args=train_args,
    masks=masks_, 
    use_cuda=True)

IN MODEL: RANDOM GENERATOR: 1


In [5]:
model.model.load_state_dict(torch.load('/home/anhnmt2/Documents/CodeInjection/date/experiments/outputs/pytorch_model.bin'))
model.model.generator_model.load_state_dict(torch.load('/home/anhnmt2/Documents/CodeInjection/date/experiments/outputs/generator_model/pytorch_model.bin'))
model.model.discriminator_model.load_state_dict(torch.load('/home/anhnmt2/Documents/CodeInjection/date/experiments/outputs/discriminator_model/pytorch_model.bin'))

<All keys matched successfully>

In [6]:
test_file = "/home/anhnmt2/Documents/CodeInjection/date/datasets/web_attack_subset/test/known.txt"
test_file_outliers = "/home/anhnmt2/Documents/CodeInjection/date/datasets/web_attack_subset/test/known-outliers.txt"
eval_dataset, eval_doc_dataset, eval_doc_dataset_outliers = model.custom_prepare_eval_data(test_file, test_file_outliers)

In [7]:
# res_out = model.custom_eval_model_anomaly(eval_doc_dataset_outliers)

inlier_sc:   0%|          | 0/286 [00:00<?, ?it/s]/home/anhnmt2/.local/lib/python3.6/site-packages/simpletransformers/language_modeling/language_modeling_utils.py:215: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = F.softmax(predict)
/home/anhnmt2/.local/lib/python3.6/site-packages/simpletransformers/language_modeling/language_modeling_utils.py:229: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = F.softmax(predict)
inlier_sc: 100%|██████████| 286/286 [00:22<00:00, 12.84it/s]


In [8]:
# res_out

{'loss': 40.46348680024381,
 'loss_rtd': 0.024745863224086645,
 'loss_rmd': 0.39226193713688556,
 'acc': 0.9681724525474527,
 'f1': 0.9837075928799457,
 'gt': array([ 0., 44., 10., 26., 22., 19., 22.,  7., 40., 43.,  8., 22.,  3.,
        24., 28., 11., 31., 49.,  0., 31., 36., 12., 45., 19., 48., 26.,
         5., 33.,  2., 30., 35.,  6., 33., 29., 31., 27.,  7., 46., 22.,
        32., 34.,  1.], dtype=float32),
 'pred': array([31, 44, 10, 26, 22, 19, 22,  7, 40, 43,  8, 22,  3, 24, 28, 11, 31,
        49, 31, 31, 36, 29, 45, 19, 48, 26,  5, 33,  2, 30, 35,  6, 33, 29,
        31, 27,  7, 46, 22, 32, 34,  1]),
 'loss_gen': 10.493164742743218}

In [9]:
# res_in = model.custom_eval_model_anomaly(eval_doc_dataset)

inlier_sc: 100%|██████████| 94/94 [00:07<00:00, 12.08it/s]


In [10]:
# res_in

{'loss': 47.556491501787875,
 'loss_rtd': 0.008511082769589538,
 'loss_rmd': 0.47130937581049637,
 'acc': 0.9311835106382979,
 'f1': 0.9641529314509574,
 'gt': array([ 4., 14.,  6., 24., 45.,  2.,  4., 25.,  7.,  2.,  0., 21., 34.,
        41., 11., 27., 33., 31., 12., 49., 42., 42., 13., 24., 37., 46.,
        31., 48.,  2., 40., 35., 20., 30., 47., 39., 18., 38., 28., 29.,
        15., 19., 22.,  9., 12.,  1., 15., 18., 26.], dtype=float32),
 'pred': array([ 4, 31,  6, 24, 45,  2,  4, 25,  7,  2,  2, 21, 34, 41, 11, 27, 33,
        31, 12, 49, 42, 42, 13, 24, 37, 46, 31, 48,  2, 40, 35, 20, 30, 47,
        32, 18, 38, 28, 29, 15, 19, 22,  9, 12,  1, 15, 18, 26]),
 'loss_gen': 10.434902018689094}

In [11]:
# with open(test_file, 'r') as f:
#     data_in = f.read().splitlines()
# with open(test_file_outliers, 'r') as f:
#     data_out = f.read().splitlines()

# #weighted f1_score
# f1_final = (len(data_in)/(len(data_in)+len(data_out)) * res_in['f1']) + (len(data_out)/(len(data_in)+len(data_out)) * res_out['f1'])
# f1_final

0.9788699246090542

In [7]:
with open(test_file) as f:
    inlier_len = len(f.readlines())
with open(test_file_outliers) as f:
    outlier_len = len(f.readlines())

In [8]:
model.inlier_len = inlier_len
model.outlier_len = outlier_len
model.global_step = 0 # Do nothing but still required
model.custom_eval_model_document(eval_doc_dataset, eval_doc_dataset_outliers)

outlier_sc_doc:   0%|          | 0/286 [00:00<?, ?it/s]/home/anhnmt2/.local/lib/python3.6/site-packages/simpletransformers/language_modeling/language_modeling_model.py:687: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out_softmax = F.softmax(d_output).cpu().numpy()
/home/anhnmt2/.local/lib/python3.6/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
inlier_sc_doc: 100%|██████████| 94/94 [00:16<00:00,  5.60it/s]


NE
ROC-AUC    : 0.697778
PR-AUC-in  : 0.412344
PR-AUC-out : 0.814464
Accuracy : 0.719457
F1-score : 0.665997
PL
ROC-AUC    : 0.697861
PR-AUC-in  : 0.412359
PR-AUC-out : 0.815058
Accuracy : 0.719540
F1-score : 0.665938
MP
ROC-AUC    : 0.697759
PR-AUC-in  : 0.412341
PR-AUC-out : 0.814337
Accuracy : 0.719540
F1-score : 0.665938
NE_RTD
ROC-AUC    : 0.836142
PR-AUC-in  : 0.627089
PR-AUC-out : 0.920035
Accuracy : 0.767770
F1-score : 0.730202
PL_RTD
ROC-AUC    : 0.545431
PR-AUC-in  : 0.317497
PR-AUC-out : 0.743339
Accuracy : 0.576588
F1-score : 0.531472
MP_RTD
ROC-AUC    : 0.839628
PR-AUC-in  : 0.628130
PR-AUC-out : 0.924393
Accuracy : 0.768235
F1-score : 0.731905


{'NE/inlier_mean': 1.9729407580296199,
 'MP/inlier_mean': 0.9543428004812449,
 'PL/inlier_mean': 0.9514241382436691,
 'NE/outlier_mean': 1.9938157511784111,
 'MP/outlier_mean': 0.992243472929818,
 'PL/outlier_mean': 0.9909524799453966,
 'NE/mean': 1.9833782546040155,
 'MP/mean': 0.9732931367055314,
 'PL/mean': 0.9711883090945328,
 'NE/roc_auc': 0.6977780501835313,
 'NE/pr_auc_in': 0.4123436482735688,
 'NE/pr_auc_out': 0.814464125775341,
 'PL/roc_auc': 0.6978605799959824,
 'PL/pr_auc_in': 0.4123592290184622,
 'PL/pr_auc_out': 0.81505784733997,
 'MP/roc_auc': 0.6977593181029602,
 'MP/pr_auc_in': 0.4123406761905741,
 'MP/pr_auc_out': 0.8143371959158469,
 'NE_RTD/roc_auc': 0.8361419839186085,
 'NE_RTD/pr_auc_in': 0.6270892952266202,
 'NE_RTD/pr_auc_out': 0.9200351662377966,
 'PL_RTD/roc_auc': 0.5454307739367045,
 'PL_RTD/pr_auc_in': 0.3174973440716895,
 'PL_RTD/pr_auc_out': 0.743338827764951,
 'MP_RTD/roc_auc': 0.8396279333077667,
 'MP_RTD/pr_auc_in': 0.6281295810055307,
 'MP_RTD/pr_auc_ou